In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [9]:
# Declare transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [10]:
# Load data and transform
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

In [7]:
# Create dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [18]:
# Build model

class MNIST_NN(nn.Module):
  def __init__(self):
    super(MNIST_NN, self).__init__()
    self.fc1 = nn.Linear(28*28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [34]:
# Init model
model = MNIST_NN()
model.train()


MNIST_NN(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [35]:
# Loss function and optimizer

ce_loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.85)

In [36]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 5

def trainingloop(num_epochs):
  for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in train_loader:
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = ce_loss(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

trainingloop(num_epochs)

Epoch 1, Loss: 1.3033
Epoch 2, Loss: 0.4432
Epoch 3, Loss: 0.3551
Epoch 4, Loss: 0.3198
Epoch 5, Loss: 0.2966


In [40]:
# Test runthrough
def nn_test():
  model.eval()
  correct = 0
  total = 0
  running_loss = 0.0

  with torch.no_grad():
    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = ce_loss(outputs, labels)
      running_loss += loss.item()


      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  avg_loss = running_loss / len(test_loader)
  accuracy = correct / total * 100

  print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
  model.train()

nn_test()

Test Loss: 0.2820, Accuracy: 91.61%
